In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
from lsst.obs.lsst import LsstCam
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
import lsst.afw.display as afwDisplay
from lsst.afw import image
from lsst.geom import Point2I
from StreakFinder_26Sep25 import find_faint_ridges

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"

# First run one CCD as a test

In [ ]:
dayObs = 20250909
seqNum = 313
expId = int(dayObs * 1E5 + seqNum)
# Can specify detNum or detName
detNum = 106
if detNum < 0:
    detName = 'R23_S21'
    det = camera[detName]
    detNum = det.getId()
print(detNum)
calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)

In [ ]:
%matplotlib inline
x = plot(calexp, stretch='ccs')
x

# Now we test the code with the different output options

In [ ]:
lines = find_faint_ridges(calexp, output="L")
print(lines)

In [ ]:
%matplotlib inline
fig = find_faint_ridges(calexp, output="P")

# The cells below run the streak finding on the whole focal plane.

In [ ]:
# This eliminates the heavily vignetted corners from the whole camera plot
rafts = [      'R01','R02','R03', 
         'R10','R11','R12','R13','R14',
         'R20','R21','R22','R23','R24',
         'R30','R31','R32','R33','R34',  
               'R41','R42','R43']
ccds = ['S00','S01','S02',
        'S10','S11','S12',
        'S20','S21','S22']
corners = ['R01_S00', 'R01_S01', 'R03_S01', 'R03_S02', \
                     'R10_S00', 'R10_S10', 'R30_S20', 'R30_S10', \
                    'R41_S20', 'R41_S21', 'R43_S21', 'R43_S22', \
                    'R34_S22', 'R34_S12', 'R14_S02', 'R14_S12']
detectorNameList = []
for raft in rafts:
    for ccd in ccds:
        name = raft+'_'+ccd
        if name not in corners:
            detectorNameList.append(name)

In [ ]:
def streakCallback(im, ccd, imageSource):
    # This runs the streak finding algorithm on each CCD
    calexp = butler.get('preliminary_visit_image', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    disp_img = find_faint_ridges(calexp, output="I")
    print(ccd.getId(), np.max(disp_img.flatten())) # This will flag which CCDs have streaks
    oim = image.ImageF(array=disp_img, deep=False, xy0=Point2I(0, 0))
    return oim

# This assembles the entire camera image

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=0, max=100)
disp.setImageColormap("gray")
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=streakCallback,
                                                       background=np.nan),
                              detectorNameList=detectorNameList,
                              binSize=16, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))